In [91]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass 
password = getpass.getpass()

········


In [127]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
query = 'SELECT r.rental_id, f.film_id, f.title, special_features, rental_rate, length, rating, store_id, r.rental_date, CASE WHEN r.rental_date LIKE "%%2005-08%%" THEN 1 ELSE 0 END AS "rented_in_Aug_2005?" FROM rental r JOIN inventory i ON r.inventory_id = i.inventory_id JOIN film f ON i.film_id = f.film_id'
data = pd.read_sql_query(query, engine)
print(data.shape)
data.head()

(16045, 10)


,rental_id,film_id,title,special_features,rental_rate,length,rating,store_id,rental_date,rented_in_Aug_2005?
0,4863,1,ACADEMY DINOSAUR,"Deleted Scenes,Behind the Scenes",0.99,86,PG,1,2005-07-08 19:03:15,0
1,11433,1,ACADEMY DINOSAUR,"Deleted Scenes,Behind the Scenes",0.99,86,PG,1,2005-08-02 20:13:10,1
2,14714,1,ACADEMY DINOSAUR,"Deleted Scenes,Behind the Scenes",0.99,86,PG,1,2005-08-21 21:27:43,1
3,16050,1,ACADEMY DINOSAUR,"Deleted Scenes,Behind the Scenes",0.99,86,PG,1,2021-08-23 15:43:01,0
4,972,1,ACADEMY DINOSAUR,"Deleted Scenes,Behind the Scenes",0.99,86,PG,1,2005-05-30 20:21:07,0


In [93]:
data.dtypes

rental_id                       int64
film_id                         int64
title                          object
special_features               object
rental_rate                   float64
length                          int64
rating                         object
store_id                        int64
rental_date            datetime64[ns]
rented_in_Aug_2005?             int64
dtype: object

In [94]:
data.isna().sum()

rental_id              0
film_id                0
title                  0
special_features       0
rental_rate            0
length                 0
rating                 0
store_id               0
rental_date            0
rented_in_Aug_2005?    0
dtype: int64

In [95]:
data['film_id'].value_counts()

103    34
738    33
489    32
382    32
331    32
       ..
335     5
107     5
904     4
400     4
584     4
Name: film_id, Length: 958, dtype: int64

In [96]:
data['length'].value_counts()

85     305
112    243
84     216
178    209
92     196
      ... 
94      42
72      42
55      42
66      22
96      20
Name: length, Length: 140, dtype: int64

In [97]:
data['rating'].value_counts()

PG-13    3585
NC-17    3293
PG       3213
R        3181
G        2773
Name: rating, dtype: int64

In [98]:
data['rental_rate'].value_counts()

0.99    5653
4.99    5272
2.99    5120
Name: rental_rate, dtype: int64

In [99]:
data['special_features'].value_counts()

Trailers,Commentaries,Behind the Scenes                   1320
Trailers                                                  1151
Trailers,Commentaries                                     1140
Trailers,Behind the Scenes                                1137
Behind the Scenes                                         1114
Commentaries,Deleted Scenes,Behind the Scenes             1112
Commentaries                                              1101
Commentaries,Behind the Scenes                            1093
Trailers,Deleted Scenes                                   1065
Deleted Scenes,Behind the Scenes                          1050
Deleted Scenes                                            1037
Commentaries,Deleted Scenes                               1020
Trailers,Commentaries,Deleted Scenes,Behind the Scenes    1003
Trailers,Commentaries,Deleted Scenes                       922
Trailers,Deleted Scenes,Behind the Scenes                  780
Name: special_features, dtype: int64

In [100]:
data['store_id'].value_counts()

2    8121
1    7924
Name: store_id, dtype: int64

In [101]:
query1 = 'SELECT f.film_id, special_features, rental_rate, length, rating, store_id, r.rental_date FROM film f JOIN inventory i ON i.film_id = f.film_id JOIN rental r ON r.inventory_id = i.inventory_id WHERE r.rental_date not LIKE "%%2005-08%%"'
data1 = pd.read_sql_query(query1, engine)
data1.shape

(10359, 7)

In [118]:
queryy = 'SELECT r.rental_id, f.film_id, f.title, r.rental_date, rental_rate FROM film f JOIN inventory i ON i.film_id = f.film_id JOIN rental r ON r.inventory_id = i.inventory_id WHERE r.rental_date LIKE "%%2005-08%%"'
datay = pd.read_sql_query(queryy, engine)
datay.head()

,rental_id,film_id,title,rental_date,rental_rate
0,10181,683,PITY BOUND,2005-08-01 00:00:44,4.99
1,10182,894,TOMATOES HELLFIGHTERS,2005-08-01 00:08:01,0.99
2,10183,941,VIDEOTAPE ARSENIC,2005-08-01 00:08:01,4.99
3,10184,611,MUSKETEERS WAIT,2005-08-01 00:09:33,4.99
4,10185,281,ENCINO ELF,2005-08-01 00:12:11,0.99


In [103]:
import numpy as np
X_num = data1.select_dtypes(include = np.number)
X_num.head()

,film_id,rental_rate,length,store_id
0,1,0.99,86,1
1,1,0.99,86,1
2,1,0.99,86,1
3,1,0.99,86,1
4,1,0.99,86,1


In [104]:
X_cat = data1.select_dtypes(include = np.object)
X_cat.head()

<ipython-input-104-657db0cdd24c>:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_cat = data1.select_dtypes(include = np.object)


,special_features,rating
0,"Deleted Scenes,Behind the Scenes",PG
1,"Deleted Scenes,Behind the Scenes",PG
2,"Deleted Scenes,Behind the Scenes",PG
3,"Deleted Scenes,Behind the Scenes",PG
4,"Deleted Scenes,Behind the Scenes",PG


In [105]:
from sklearn.preprocessing import MinMaxScaler
transformer = MinMaxScaler().fit(X_num)
x_normalized = transformer.transform(X_num)
print(x_normalized.shape)
pd.DataFrame(x_normalized, columns=X_num.columns)

(10359, 4)


,film_id,rental_rate,length,store_id
0,0.0,0.0,0.287770,0.0
1,0.0,0.0,0.287770,0.0
2,0.0,0.0,0.287770,0.0
3,0.0,0.0,0.287770,0.0
4,0.0,0.0,0.287770,0.0
...,...,...,...,...
10354,1.0,1.0,0.028777,1.0
10355,1.0,1.0,0.028777,1.0
10356,1.0,1.0,0.028777,1.0
10357,1.0,1.0,0.028777,1.0


In [112]:
x_normalized.head()

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [107]:
categorical = pd.get_dummies(X_cat, 
                             columns=['special_features', 'rating'],
                             drop_first=True)
categorical.head()

,special_features_Commentaries,"special_features_Commentaries,Behind the Scenes","special_features_Commentaries,Deleted Scenes","special_features_Commentaries,Deleted Scenes,Behind the Scenes",special_features_Deleted Scenes,"special_features_Deleted Scenes,Behind the Scenes",special_features_Trailers,"special_features_Trailers,Behind the Scenes","special_features_Trailers,Commentaries","special_features_Trailers,Commentaries,Behind the Scenes","special_features_Trailers,Commentaries,Deleted Scenes","special_features_Trailers,Commentaries,Deleted Scenes,Behind the Scenes","special_features_Trailers,Deleted Scenes","special_features_Trailers,Deleted Scenes,Behind the Scenes",rating_NC-17,rating_PG,rating_PG-13,rating_R
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0


In [114]:
X = pd.concat([X_num, categorical], axis=1)

In [115]:
X.head()

,film_id,rental_rate,length,store_id,special_features_Commentaries,"special_features_Commentaries,Behind the Scenes","special_features_Commentaries,Deleted Scenes","special_features_Commentaries,Deleted Scenes,Behind the Scenes",special_features_Deleted Scenes,"special_features_Deleted Scenes,Behind the Scenes",...,"special_features_Trailers,Commentaries","special_features_Trailers,Commentaries,Behind the Scenes","special_features_Trailers,Commentaries,Deleted Scenes","special_features_Trailers,Commentaries,Deleted Scenes,Behind the Scenes","special_features_Trailers,Deleted Scenes","special_features_Trailers,Deleted Scenes,Behind the Scenes",rating_NC-17,rating_PG,rating_PG-13,rating_R
0,1,0.99,86,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,1,0.99,86,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,1,0.99,86,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,1,0.99,86,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,1,0.99,86,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [121]:
y = data1['rental_date']

In [122]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [125]:
from sklearn.linear_model import LogisticRegression
classification = LogisticRegression(random_state=0, solver='saga',
                  multi_class='multinomial').fit(X_train, y_train)

C:\Users\hp\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [128]:
predictions = classification.predict(X_test)
classification.score(X_test, y_test)

0.014800514800514801

In [ ]:
## i have messed i will continue to work on my model
